In [1]:
import maxim_pred as maxim
import IAGC
import cv2
import numpy as np
import eval_metrics
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import matlab.engine
import random
from medpy.filter.smoothing import anisotropic_diffusion
import tensorflow as tf

In [18]:
def calc_eval_metrics(clean_data, noisy_data, agc_corrected_data, max_corrected_data, ad_corrected_data):
    pre_psnr = []
    post_psnr = []
    pre_ssim = []
    post_ssim = []
    pre_cwssim = []
    post_cwssim = []
    pre_unique = []
    post_unique = []
    pre_msunique = []
    post_msunique = []
    pre_summer = []
    post_summer = []
    pre_csv = []
    post_csv = []

    post_psnr1 = []
    post_ssim1 = []
    post_cwssim1 = []
    post_unique1 = []
    post_msunique1 = []
    post_summer1 = []
    post_csv1 = []

    post_psnr2 = []
    post_ssim2 = []
    post_cwssim2 = []
    post_unique2 = []
    post_msunique2 = []
    post_summer2 = []
    post_csv2 = []


    print('Calculating evaluation metrics ...')
    for i in tqdm(range(len(clean_data)), desc='processing'):
        cimg = clean_data[i]
        nimg = noisy_data[i]
        y_agc = agc_corrected_data[i]
        y_max = max_corrected_data[i]
        y_ad = ad_corrected_data[i]
    

        m_cimg = matlab.double(cimg.tolist())
        m_nimg = matlab.double(nimg.tolist())
        m_y_agc = matlab.double(y_agc.tolist())
        m_y_max = matlab.double(y_max.tolist())
        m_y_ad = matlab.double(y_ad.tolist())

        # Convert BGR to RGB
        cimg = cv2.cvtColor(cimg, cv2.COLOR_BGR2RGB)
        nimg = cv2.cvtColor(nimg, cv2.COLOR_BGR2RGB)
        
        
        pre_psnr.append(eval_metrics.PSNR(cimg, nimg))
        pre_ssim.append(eval_metrics.calculate_ssim(cimg, nimg))
        pre_cwssim.append(eval_metrics.calculate_cwssim(cimg, nimg))
        pre_summer.append(eval_metrics.calculate_summer(cimg, nimg))
        unique_val = eng.mslUNIQUE(m_cimg, m_nimg)
        pre_unique.append(np.array(unique_val))
        msunique_val = eng.mslMSUNIQUE(m_cimg, m_nimg)
        pre_msunique.append(np.array(msunique_val))
        csv_val = eng.csv(m_cimg, m_nimg)
        pre_csv.append(np.array(csv_val))

        post_psnr.append(eval_metrics.PSNR(cimg, y_agc))
        post_ssim.append(eval_metrics.calculate_ssim(cimg, y_agc))
        post_cwssim.append(eval_metrics.calculate_cwssim(cimg, y_agc))
        post_summer.append(eval_metrics.calculate_summer(cimg, y_agc))
        unique_val = eng.mslUNIQUE(m_cimg, m_y_agc)
        post_unique.append(np.array(unique_val))
        msunique_val = eng.mslMSUNIQUE(m_cimg, m_y_agc)
        post_msunique.append(np.array(msunique_val))
        csv_val = eng.csv(m_cimg, m_y_agc)
        post_csv.append(np.array(csv_val))

        post_psnr1.append(eval_metrics.PSNR(cimg, y_max))
        post_ssim1.append(eval_metrics.calculate_ssim(cimg, y_max))
        post_cwssim1.append(eval_metrics.calculate_cwssim(cimg, y_max))
        post_summer1.append(eval_metrics.calculate_summer(cimg, y_max))
        unique_val = eng.mslUNIQUE(m_cimg, m_y_max)
        post_unique1.append(np.array(unique_val))
        msunique_val = eng.mslMSUNIQUE(m_cimg, m_y_max)
        post_msunique1.append(np.array(msunique_val))
        csv_val = eng.csv(m_cimg, m_y_max)
        post_csv1.append(np.array(csv_val))

        post_psnr2.append(eval_metrics.PSNR(cimg, y_ad))
        post_ssim2.append(eval_metrics.calculate_ssim(cimg, y_ad))
        post_cwssim2.append(eval_metrics.calculate_cwssim(cimg, y_ad))
        post_summer2.append(eval_metrics.calculate_summer(cimg, y_ad))
        unique_val = eng.mslUNIQUE(m_cimg, m_y_ad)
        post_unique2.append(np.array(unique_val))
        msunique_val = eng.mslMSUNIQUE(m_cimg, m_y_ad)
        post_msunique2.append(np.array(msunique_val))
        csv_val = eng.csv(m_cimg, m_y_ad)
        post_csv2.append(np.array(csv_val))

    metrics = ['PSNR', 'SSIM', 'CWSSIM', 'UNIQUE', 'MSUNIQUE', 'SUMMER', 'CSV']
    mean_values_initial = []
    for i, lst in enumerate([pre_psnr, pre_ssim, pre_cwssim, pre_unique, pre_msunique, pre_summer, pre_csv]):
        mean_value = np.mean(lst)
        mean_values_initial.append([f"{mean_value:.3f}"])

    # Display the mean values after adding 10 to the first element
    mean_values_after_agc = []
    for i, lst in enumerate([post_psnr, post_ssim, post_cwssim, post_unique, post_msunique, post_summer, post_csv]):
        mean_value = np.mean(lst)
        mean_values_after_agc.append([f"{mean_value:.3f}"])

    # Display the mean values after adding 10 to the first element
    mean_values_after_max = []
    for i, lst in enumerate([post_psnr1, post_ssim1, post_cwssim1, post_unique1, post_msunique1, post_summer1, post_csv1]):
        mean_value = np.mean(lst)
        mean_values_after_max.append([f"{mean_value:.3f}"])

    # Display the mean values after adding 10 to the first element
    mean_values_after_ad = []
    for i, lst in enumerate([post_psnr2, post_ssim2, post_cwssim2, post_unique2, post_msunique2, post_summer2, post_csv2]):
        mean_value = np.mean(lst)
        mean_values_after_ad.append([f"{mean_value:.3f}"])

    return metrics, mean_values_initial, mean_values_after_agc, mean_values_after_max, mean_values_after_ad


In [3]:
# cure-OR evaluation
SIZE = 256
path = 'D:/Pranav/Data/CURE-OR/input/challenge-free'
clean_files = sorted([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and not f.startswith('.')])
idx = random.sample(range(len(clean_files)), 10)
selected_clean_files = [clean_files[i] for i in idx]
clean_data = []
print('Loading clean data ...')
for i in tqdm(selected_clean_files):
    img = cv2.imread(path + '/' + i, 1)  # Change 0 to 1 for color images
    img = cv2.resize(img, (SIZE, SIZE))
    clean_data.append(np.array(img))

Loading clean data ...


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 29.07it/s]


In [4]:
directory_path = 'D:/Pranav/Data/CURE-OR/input/'
# Get a list of folder names in the directory
noise_types = [folder for folder in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, folder))]
noise_types.remove('challenge-free')
eng = matlab.engine.start_matlab()
print(noise_types)

['underexposure-level1', 'underexposure-level2', 'underexposure-level3', 'underexposure-level4', 'overexposure-level1', 'overexposure-level2', 'overexposure-level3', 'overexposure-level4', 'blur-level1', 'blur-level2', 'blur-level3', 'blur-level4', 'contrast-level1', 'contrast-level2', 'contrast-level3', 'contrast-level4', 'dirtylens-1-level1', 'dirtylens-1-level2', 'dirtylens-1-level3', 'dirtylens-1-level4', 'dirtylens-2-level1', 'dirtylens-2-level2', 'dirtylens-2-level3', 'dirtylens-2-level4', 'salt&pepper-level1', 'salt&pepper-level2', 'salt&pepper-level3', 'salt&pepper-level4', 'resize-level1', 'resize-level2', 'resize-level3', 'resize-level4']


In [5]:
enhancement_model = tf.keras.models.load_model('D:/Pranav/Code/maxim-tf-main/models/Enhancement')
deblurring_model = tf.keras.models.load_model('D:/Pranav/Code/maxim-tf-main/models/Deblurring')
deraining_model = tf.keras.models.load_model('D:/Pranav/Code/maxim-tf-main/models/Deraining')
dehazing_model = tf.keras.models.load_model('D:/Pranav/Code/maxim-tf-main/models/Dehazing')
denoising_model = tf.keras.models.load_model('D:/Pranav/Code/maxim-tf-main/models/Denoising')

In [5]:
import pandas as pd
df_agc = pd.DataFrame(columns=['Noise Type', 'PSNR', 'SSIM', 'CW-SSIM', 'UNIQUE', 'MSUNIQUE', 'SUMMER', 'CSV'])
df_max = pd.DataFrame(columns=['Noise Type', 'PSNR', 'SSIM', 'CW-SSIM', 'UNIQUE', 'MSUNIQUE', 'SUMMER', 'CSV'])
df_ad = pd.DataFrame(columns=['Noise Type', 'PSNR', 'SSIM', 'CW-SSIM', 'UNIQUE', 'MSUNIQUE', 'SUMMER', 'CSV'])

In [6]:
import skimage.io as io
import skimage.filters as flt
%matplotlib inline
# since we can't use imports
import numpy as np
import scipy.ndimage.filters as flt
import warnings

def anisodiff(img,niter=1,kappa=50,gamma=0.1,step=(1.,1.),sigma=0, option=1,ploton=False):
	# really want
	if img.ndim == 3:
		warnings.warn("Only grayscale images allowed, converting to 2D matrix")
		img = img.mean(2)

	# initialize output array
	img = img.astype('float32')
	imgout = img.copy()

	# initialize some internal variables
	deltaS = np.zeros_like(imgout)
	deltaE = deltaS.copy()
	NS = deltaS.copy()
	EW = deltaS.copy()
	gS = np.ones_like(imgout)
	gE = gS.copy()

	# create the plot figure, if requested
	if ploton:
		import pylab as pl
		from time import sleep

		fig = pl.figure(figsize=(20,5.5),num="Anisotropic diffusion")
		ax1,ax2 = fig.add_subplot(1,2,1),fig.add_subplot(1,2,2)

		ax1.imshow(img,interpolation='nearest')
		ih = ax2.imshow(imgout,interpolation='nearest',animated=True)
		ax1.set_title("Original image")
		ax2.set_title("Iteration 0")

		fig.canvas.draw()

	for ii in np.arange(1,niter):

		# calculate the diffs
		deltaS[:-1,: ] = np.diff(imgout,axis=0)
		deltaE[: ,:-1] = np.diff(imgout,axis=1)

		if 0<sigma:
			deltaSf=flt.gaussian_filter(deltaS,sigma);
			deltaEf=flt.gaussian_filter(deltaE,sigma);
		else: 
			deltaSf=deltaS;
			deltaEf=deltaE;
			
		# conduction gradients (only need to compute one per dim!)
		if option == 1:
			gS = np.exp(-(deltaSf/kappa)**2.)/step[0]
			gE = np.exp(-(deltaEf/kappa)**2.)/step[1]
		elif option == 2:
			gS = 1./(1.+(deltaSf/kappa)**2.)/step[0]
			gE = 1./(1.+(deltaEf/kappa)**2.)/step[1]

		# update matrices
		E = gE*deltaE
		S = gS*deltaS

		# subtract a copy that has been shifted 'North/West' by one
		# pixel. don't as questions. just do it. trust me.
		NS[:] = S
		EW[:] = E
		NS[1:,:] -= S[:-1,:]
		EW[:,1:] -= E[:,:-1]

		# update the image
		imgout += gamma*(NS+EW)

		if ploton:
			iterstring = "Iteration %i" %(ii+1)
			ih.set_data(imgout)
			ax2.set_title(iterstring)
			fig.canvas.draw()
			# sleep(0.01)

	return imgout

In [7]:
def anisotropic_diffusion_func(image):
    # Split the color image into its RGB channels
    alpha = 0.1  # Diffusion factor
    K = 30       # Perona-Malik coefficient
    niters = 10  # Number of iterations
    b, g, r = cv2.split(image)

    # Apply anisotropic diffusion to each channel
    b_filtered = anisodiff(b,kappa=30)
    g_filtered = anisodiff(g,kappa=30)
    r_filtered = anisodiff(r,kappa=30)

    # Merge the filtered channels back into a color image
    filtered_image = cv2.merge([b_filtered, g_filtered, r_filtered])
    return filtered_image

In [9]:
def denoise(model, img):
    # Read the input image
#input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    input_img= img.astype(np.float32)/ 255.0
    # Process the image
    processed_img = _process_image(input_img, model)

    return processed_img

def _process_image(input_img, model):
    # Resize the input image
    image = tf.convert_to_tensor(input_img)
    
    # Handle multi-stage outputs, obtain the last scale output of the last stage
    preds = model.predict(tf.expand_dims(image, axis=0))
    if isinstance(preds,list):
        preds = preds[-1]
        if isinstance(preds,list):
            preds = preds[-1]
    preds = np.array(preds[0], np.float32)*255.0
    preds = (preds - np.min(preds))/(np.max(preds)- np.min(preds)) * 255
    preds = preds.astype(np.uint8)
    
    return preds

In [20]:
for noise_type in noise_types:
    noisy_data = []
    agc_corrected_data = []
    max_corrected_data = []
    ad_corrected_data = []
    
    noise_name = noise_type.split('-')[0]
    print("Noise Type: " + noise_name)
    
    if "exposure" in noise_type.lower() or "contrast" in noise_type.lower() or "shadow" in noise_type.lower() or "dark" in noise_type.lower():
        model = enhancement_model
    elif "blur" in noise_type.lower():
        model = deblurring_model
    elif "rain" in noise_type.lower() or "snow" in noise_type.lower():
        model = deraining_model
    elif "haz" in noise_type.lower():
        model = dehazing_model
    else:
        model = denoising_model
    
    path = 'D:/Pranav/Data/CURE-OR/input/' + noise_type
    save_dir = 'D:/Pranav/Data/CURE-OR/output/' + noise_type
    noisy_files = sorted([f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and not f.startswith('.')])
    selected_noisy_files = [noisy_files[i] for i in idx]
    for i in tqdm(selected_noisy_files):
        img = cv2.imread(path + '/' + i, 1)  # Change 0 to 1 for color images
        img = cv2.resize(img, (SIZE, SIZE))
        if not os.path.exists(save_dir + '/Gamma Correction'):
            os.makedirs(save_dir + '/Gamma Correction')
            os.makedirs(save_dir + '/Maxim')
            os.makedirs(save_dir + '/Anisotropic Diffusion')

        op_agc = IAGC.adaptive_gamma_correction(img)
        op_max = denoise(model, img)
        op_ad = anisotropic_diffusion_func(img)
        cv2.imwrite(save_dir + '/Gamma Correction/' + i + '.png',op_agc)
        cv2.imwrite(save_dir + '/Maxim/' + i + '.png',op_max)
        cv2.imwrite(save_dir + '/Anisotropic Diffusion/' + i + '.png',op_ad)
        noisy_data.append(np.array(img))
        agc_corrected_data.append(np.array(op_agc))
        max_corrected_data.append(np.array(op_max))
        ad_corrected_data.append(np.array(op_ad))
        
    metrics, mean_values_initial, mean_values_after_agc, mean_values_after_max, mean_values_after_ad = calc_eval_metrics(clean_data, noisy_data, agc_corrected_data, max_corrected_data, ad_corrected_data)

    df_agc = df_agc._append({'Noise Type': noise_type, 'PSNR': f'{mean_values_initial[0][0]}/{mean_values_after_agc[0][0]}', 
                                'SSIM': f'{mean_values_initial[1][0]}/{mean_values_after_agc[1][0]}', 
                                'CW-SSIM': f'{mean_values_initial[2][0]}/{mean_values_after_agc[2][0]}', 
                                'UNIQUE': f'{mean_values_initial[3][0]}/{mean_values_after_agc[3][0]}',
                                'MSUNIQUE': f'{mean_values_initial[4][0]}/{mean_values_after_agc[4][0]}',
                                'SUMMER': f'{mean_values_initial[5][0]}/{mean_values_after_agc[5][0]}',
                                'CSV': f'{mean_values_initial[6][0]}/{mean_values_after_agc[6][0]}'}, ignore_index=True)
    
    df_max = df_max._append({'Noise Type': noise_type, 'PSNR': f'{mean_values_initial[0][0]}/{mean_values_after_max[0][0]}', 
                                'SSIM': f'{mean_values_initial[1][0]}/{mean_values_after_max[1][0]}', 
                                'CW-SSIM': f'{mean_values_initial[2][0]}/{mean_values_after_max[2][0]}', 
                                'UNIQUE': f'{mean_values_initial[3][0]}/{mean_values_after_max[3][0]}',
                                'MSUNIQUE': f'{mean_values_initial[4][0]}/{mean_values_after_max[4][0]}',
                                'SUMMER': f'{mean_values_initial[5][0]}/{mean_values_after_max[5][0]}',
                                'CSV': f'{mean_values_initial[6][0]}/{mean_values_after_max[6][0]}'}, ignore_index=True)
    
    df_ad = df_ad._append({'Noise Type': noise_type, 'PSNR': f'{mean_values_initial[0][0]}/{mean_values_after_ad[0][0]}', 
                                'SSIM': f'{mean_values_initial[1][0]}/{mean_values_after_ad[1][0]}', 
                                'CW-SSIM': f'{mean_values_initial[2][0]}/{mean_values_after_ad[2][0]}', 
                                'UNIQUE': f'{mean_values_initial[3][0]}/{mean_values_after_ad[3][0]}',
                                'MSUNIQUE': f'{mean_values_initial[4][0]}/{mean_values_after_ad[4][0]}',
                                'SUMMER': f'{mean_values_initial[5][0]}/{mean_values_after_ad[5][0]}',
                                'CSV': f'{mean_values_initial[6][0]}/{mean_values_after_ad[6][0]}'}, ignore_index=True)
    
    
    path = "D:/Pranav/Results/CURE-OR"
    if not os.path.exists(path):
        os.makedirs(path)
    
    # Save the table to a file
    filename = path+'/'+ "AGC.csv"
    df_agc.to_csv(filename, index=False)
    
    filename = path+'/'+ "MAXIM.csv"
    df_max.to_csv(filename, index=False)
    
    filename = path+'/'+ "AD.csv"
    df_ad.to_csv(filename, index=False)

Noise Type: underexposure


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  8.73it/s]


Calculating evaluation metrics ...


processing: 100%|██████████████████████████████████████████████████████████████████████| 10/10 [00:27<00:00,  2.73s/it]


Noise Type: underexposure


...

processing: 100%|██████████████████████████████████████████████████████████████████████| 10/10 [00:27<00:00,  2.75s/it]


Noise Type: resize


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.66it/s]


Calculating evaluation metrics ...


processing: 100%|██████████████████████████████████████████████████████████████████████| 10/10 [00:30<00:00,  3.05s/it]


In [11]:
eng.quit()